In [ ]:
# |default_exp core
# This will create a package named $PACKAGE_NAME/core.py

# Global static vars
These are used to modify the template for individual use cases

In [ ]:
# |export
# Need the $PACKAGE_NAME for a few functions, this can be considered a static var
PACKAGE_NAME: str = "$PACKAGE_NAME"  # Make sure to adjust this to your package name
import os
# This is the root folder of the project, we will use this to find the package folder, its also a notebook so __file__ will not work, we're in nbs which is one above the root
PROJECT_FOLDER: str = os.path.abspath(os.path.join(os.getcwd(), ".."))

DEV_MODE: bool = False
# if the folder template_nbdev_example exists in the cwd, we are in dev mode. Probably want a better way to determine this going forward
if os.path.exists(f"{PROJECT_FOLDER}/{PACKAGE_NAME}"):
    DEV_MODE = True

For help with the Markdown language, see [this guide](https://www.markdownguide.org/basic-syntax/).

In [ ]:
# |hide
import nbdev
from nbdev.showdoc import *  # ignore this Pylance warning in favor of following nbdev docs

# Core

 A module which contains common functions to be used by other modules. Those that exist in the template are meant to be common functions we can use against multiple packages.

#|hide

Notebook blocks starting with #|hide are not shown in the documentation and not exported to the python package. Blocks with #|export are exported to the python package. Blocks with neither are shown to the documentation but not exported to the python package.

## Libraries

Currently all libraries included are listed at the top and calls to them are also made in the block of code that uses them. This is for readability and the performance hit of the import is negligible.

In [ ]:
# |export
# standard libs
import os
import re

# Common to template
# add into settings.ini, requirements, package name is python-dotenv, for conda build ensure `conda config --add channels conda-forge`
import dotenv  # for loading config from .env files, https://pypi.org/project/python-dotenv/
import envyaml  # Allows to loads env vars into a yaml file, https://github.com/thesimj/envyaml
import fastcore  # To add functionality related to nbdev development, https://github.com/fastai/fastcore/
from fastcore import (
    test,
)
from fastcore.script import (
    call_parse,
)  # for @call_parse, https://fastcore.fast.ai/script

# Project specific libraries

## Config

Our config file holds all program and user specific variables. This is a good practice to follow as it allows us to easily change variables without having to change code. It also allows us to easily change variables based on the environment we are running in. For example, we may want to run a program in a test environment with a different database than we would in production. This is also a good practice to follow as it allows us to easily change variables without having to change code. It also allows us to easily change variables based on the environment we are running in. For example, we may want to run a program in a test environment with a different database than we would in production.

Configuration is templated to rely on environment (ENV) variables. A default ENV config is provided in `./config/config.default.env` and more advanced data structures are supported in `./config/config.default.yaml`. The `.yaml` file is meant to represent what your program actually works with and the `.env` file options the user can change at run time.

Make sure you know the priority of variables and check on them when debugging your code. Also ensure that your yaml file is referenced appropriately in the `.env` file. 

When in use there's an expectation you'll have multiple config files for different use cases e.g. development, production environment for different paths, etc.

### set env variables
A helper function for getting your config values, this will set the environment variables with the provided `.env` values. If you're missing values it'll ensure they're loaded in with the defaults file.

In [ ]:
# |export
import importlib
import importlib.util
def set_env_variables(config_path: str, overide_env_vars: bool = True) -> bool:
    # Load dot env sets environmental values from a file, if the value already exists it will not be overwritten unless override is set to True.
    # If we have multiple .env files then we need to apply the one which we want to take precedence last with overide.

    # Order of precedence: .env file > environment variables > default values
    # When developing, making a change to the config will not be reflected until the environment is restarted

    # Set the env vars first, this is needed for the card.yaml to replace ENV variables
    # NOTE: You need to adjust PROJECT_NAME to your package name for this to work, the exception is only for dev purposes
    # This here checks if your package is installed, such as through pypi or through pip install -e  [.dev] for development. If it is then it'll go there and use the config files there as your default values.
    spec = importlib.util.find_spec(PACKAGE_NAME)
    if DEV_MODE:
        # Check if {PACKAGE_NAME}/config/config.default.env exists relative to the current working directory
        if os.path.isfile(f"{PROJECT_FOLDER}/{PACKAGE_NAME}/config/config.default.env"):
            dotenv.load_dotenv(
                f"{PROJECT_FOLDER}/{PACKAGE_NAME}/config/config.default.env",
                override=False,
            )
            print(f"Loaded {PROJECT_FOLDER}/{PACKAGE_NAME}/config/config.default.env")
        else:
            print(
                f"Error: {PACKAGE_NAME}/config/config.default.env does not exist in the current working directory {PROJECT_FOLDER}"
            )
            return False
    elif spec is not None:
        module = importlib.util.module_from_spec(spec)
        # Check that the config exists in the loaded module, it should be in config/config.default.env
        if os.path.isfile(f"{spec.origin}/{PACKAGE_NAME}/config/config.default.env"):
            dotenv.load_dotenv(f"{spec.origin}/{PACKAGE_NAME}/config/config.default.env", override=False)
        else:
            # Here it means the default in the package. This could mean we're in development mode.
            print(
                f"Error: {PACKAGE_NAME}/config/config.default.env does not exist in the package {PACKAGE_NAME} "
            )
            return False
    # 2. set values from file:
    if os.path.isfile(config_path):
        dotenv.load_dotenv(config_path, override=overide_env_vars)

    return True

IndentationError: expected an indented block after 'if' statement on line 15 (2423656242.py, line 16)

### get config

When you run this function, assuming things are set up properly, you end up with a dict that matches your `.yaml` file. This file will have all the inputs for the package and settings of your program.

To do this it will use a `.env` config file, which has an associated yaml file defined with `CORE_YAML_CONFIG_FILE` in the `.env` file. And then use the `.env` file to load values into the associated `.yaml` file.

In [ ]:
# |export


def get_config(config_path: str = None, overide_env_vars: bool = True) -> dict:
    if config_path is None:
        config_path = ""
    # First sets environment with variables from config_path, then uses those variables to fill in appropriate values in the config.yaml file, the yaml file is then returned as a dict
    # If you want user env variables to take precedence over the config.yaml file then set overide_env_vars to False
    set_env_variables(config_path, overide_env_vars)
    config: dict = envyaml.EnvYAML(
        os.environ.get(
            "CORE_YAML_CONFIG_FILE", f"./{PACKAGE_NAME}/config/config.default.yaml"
        ),
        strict=False,
    ).export()
    return config

### Variables

All the user input variables and machine adjustable variables should be in your config, which is a dict. Reference config.default.yaml for how to access your variables. Also note that with python dicts you can use `dict_variable.get("variable", default_value)` to ensure that you don't get a key error if the variable is not set.

In [ ]:
# |export
# create a os.PathLike object
config = get_config(os.environ.get("CORE_CONFIG_FILE", ""))

Error: template_nbdev_example/config/config.default.env does not exist in the package template_nbdev_example


### show project env vars
A helper function intended to only be used with debugging. It shows all your project specific environmental variables.

In [ ]:
# |export
def show_project_env_vars(config: dict) -> None:
    # Prints out all the project environment variables
    # This is useful for debugging and seeing what is being set
    for k, v in config.items():
        # If ENV var starts with PROJECTNAME_ then print
        if k.startswith(config["CORE_PROJECT_VARIABLE_PREFIX"]):
            print(f"{k}={v}")

In [ ]:
# |hide
show_project_env_vars(config)

KeyError: 'CORE_PROJECT_VARIABLE_PREFIX'

The functions below are **not** tempalted and you should adjust this with your own code. It's included as an example of how to code some functions with associated tests and how to make it work on the command line. It is best to code by creating a new workbook and then importing the functions of this into that one.

In [ ]:
# |export


def hello_world(name: str = "Not given") -> str:
    return f"Hello World! My name is {name}"

This here is a a test as part of fastcore.test, all fastcore tests will be automatically run when doing nbdev_test as well as through github actions.

In [ ]:
PROJECT_FOLDER

'/Users/kimn/git.repositories/ssi-dk/template-nbdev'

In [ ]:
test.test_eq("Hello World! My name is Kim", hello_world("Kim"))

The @call_parse will, with the settings.ini entry way, automatically transform your function into a command line tool. Comments of the functions will appear for help messages and the initial docstring will appear in the help as well. You can also define defaults for the arguments and should define a typehint to control inputs. The function will likely have to resolve variables with ENV vars and config files. The recommended way to do this is to assume variables passed here are a higher priority.

In [ ]:
# |export
from fastcore.script import call_parse


@call_parse
def cli(
    name: str,  # Your name
    config_file: str = None,  # config file to set env vars from
) -> bool:
    """
    This will print Hello World! with your name
    """
    config = get_config(config_file)  # Set env vars and get config variables
    if name is not None:
        config["example"]["user_input"]["name"] = name
    print(hello_world(name))
    return True

Test the function with a known input

In [ ]:
test.test_eq(True, cli("Kim"))

Test the function with potentially variable input to confirm output

In [ ]:
cli(config["example"]["user_input"]["name"])
cli(config["example"]["user_input"]["alternative_name"])

In [ ]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the associated python package
import nbdev 
nbdev.nbdev_export()